In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
stock_list = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
stockSH = stock_list[[i[:2] == 'SH' for i in stock_list]]
stockSZ = stock_list[[i[:2] == 'SZ' for i in stock_list]]
stockSH = [int(i[2:]) for i in stockSH]
stockSZ = [int(i[2:]) for i in stockSZ]
display(stockSH)
display(len(stock_list))
display(len(stockSH))
display(len(stockSZ))

[603693,
 600707,
 603700,
 688268,
 603059,
 600006,
 603990,
 600518,
 600521,
 600513,
 601336,
 603283,
 600714,
 601128,
 600429,
 600118,
 688168,
 603181,
 600837,
 603003,
 600683,
 603588,
 600566,
 601577,
 603711,
 600688,
 600133,
 600857,
 600207,
 603336,
 688099,
 603353,
 603787,
 603538,
 601866,
 603587,
 601155,
 603369,
 601816,
 603639,
 600706,
 600720,
 688128,
 600293,
 600383,
 603888,
 600377,
 600804,
 603499,
 600385,
 603356,
 603297,
 603729,
 600186,
 603110,
 603055,
 600919,
 601211,
 600378,
 603118,
 600612,
 603697,
 603203,
 600068,
 603895,
 600308,
 688181,
 600126,
 600827,
 600064,
 600199,
 688138,
 600939,
 600120,
 688033,
 600150,
 603969,
 601313,
 603339,
 603028,
 600449,
 603599,
 603330,
 600681,
 600756,
 603332,
 600259,
 601618,
 600338,
 600292,
 600965,
 600989,
 600819,
 603615,
 600446,
 600814,
 600081,
 600826,
 603985,
 600172,
 600663,
 600104,
 601339,
 603889,
 603987,
 600701,
 600112,
 600592,
 600350,
 601021,
 601588,
 

3839

1611

2228

# Orignal Version

In [3]:
columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]

year = "2019"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs[247:]

array(['J:\\LEVEL2_shenzhen\\2019\\1225',
       'J:\\LEVEL2_shenzhen\\2019\\1226',
       'J:\\LEVEL2_shenzhen\\2019\\1227',
       'J:\\LEVEL2_shenzhen\\2019\\1230',
       'J:\\LEVEL2_shenzhen\\2019\\1231'], dtype='<U35')

In [4]:
columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]

year = "2019"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[247:]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
        
    # am & pm
    if len(data) == 28:
        path = r'E:\SZ\2019' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        os.system("copy /b am_snap_level_spot.7z.* am_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        os.system("copy /b pm_snap_level_spot.7z.* pm_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()

        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1
        am_snap2 = pd.read_table(path1 + "\\am_snap_level_spot.txt",header=None)
        assert(am_snap2.shape[1] == len(columns2))
        am_snap2.columns = columns2
        am_snap2["time"] = am_snap2["OrigTime"] - 20190315000000000
        dd1 = int(am_snap2["OrigTime"].iloc[0]//1000000000*1000000000) + 93000000
        dd2 = int(am_snap2["OrigTime"].iloc[0]//1000000000*1000000000) + 150000000
        am = pd.merge(am_snap2, am_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"], how="outer")
        try:
            assert((am.shape[0] == am_snap2.shape[0]) & (am.shape[0] == am_snap1.shape[0]) & (am_snap1.shape[0] == am_snap2.shape[0]))
        except:
            if am.shape[0] == am_snap1.shape[0]:
                print("am_snap1 have more ticks than am_snap2")
                if all(am[(am["dbtime_x"].isnull()) & ((am["SecurityID"] < 4000) | (am["SecurityID"] > 200000))]["OrigTime"].unique() < dd1):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens after 9:30")      
            if am.shape[0] == am_snap2.shape[0]:
                print("am_snap2 have more ticks than am_snap1")
                if all(am[(am["dbtime_y"].isnull()) & ((am["SecurityID"] < 4000) | (am["SecurityID"] > 200000))]["OrigTime"].unique() < dd1):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens after 9:30")
            else:
                print("am_snap2 don't join with am_snap1")
            am = pd.merge(am_snap2, am_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"])       
        del am_snap1
        del am_snap2
        
        
        
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        pm_snap2 = pd.read_table(path1 + "\\pm_snap_level_spot.txt",header=None)
        assert(pm_snap2.shape[1] == len(columns2))
        pm_snap2.columns = columns2
        pm_snap2["time"] = pm_snap2["OrigTime"] - 20190315000000000
        pm = pd.merge(pm_snap2, pm_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"], how="outer")
        try:
            assert((pm.shape[0] == pm_snap2.shape[0]) & (pm.shape[0] == pm_snap1.shape[0]) & (pm_snap1.shape[0] == pm_snap2.shape[0]))
        except:
            if pm.shape[0] == pm_snap1.shape[0]:
                print("pm_snap1 have more ticks than pm_snap2")
                if all(pm[(pm["dbtime_x"].isnull()) & ((pm["SecurityID"] < 4000) | (pm["SecurityID"] > 200000))]["OrigTime"].unique() > dd2):
                    print("More ticks happens after 15:00")
                else:
                    print("There are ticks happens before 15:00")
            if pm.shape[0] == pm_snap2.shape[0]:
                print("pm_snap2 have more ticks than pm_snap1")
                if all(pm[(pm["dbtime_y"].isnull()) & ((pm["SecurityID"] < 4000) | (pm["SecurityID"] > 200000))]["OrigTime"].unique() > dd2):
                    print("More ticks happens after 15:00")
                else:
                    print("There are ticks happens before 15:00")
            else:
                print("pm_snap2 don't join with pm_snap1")
            pm = pd.merge(pm_snap2, pm_snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"])   
        del pm_snap1
        del pm_snap2
        snapshot = pd.concat([am, pm]).sort_values(by=["SecurityID", "OrigTime"])
        del am
        del pm
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - 20190315000000000).astype(int)
        
        print("am & pm data")
   
    # 深交所数据
    else:
        path = r'E:\SZ\2019' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\hq_snap.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\hq_snap.7z')
            bad.append(data + '\\hq_snap.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\snap_level.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\snap_level.7z')
            bad.append(data + '\\snap_level.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        snap1 = pd.read_table(path1 + "\\hq_snap.txt", header=None, encoding="UTF-8-sig")
        assert(snap1.shape[1] == len(columns1))
        snap1.columns = columns1
        snap2 = pd.read_table(path1 + "\\snap_level.txt",header=None, encoding="UTF-8-sig")
        assert(snap2.shape[1] == len(columns2))
        snap2.columns = columns2
        snap2["SecurityID"] = snap2["SecurityID"].astype(int)
        snap1["SecurityID"] = snap1["SecurityID"].astype(int)
        snap2["time"] = snap2["OrigTime"] - 20190315000000000
        snapshot = pd.merge(snap2, snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"], how="outer")
        try:
            assert((snapshot.shape[0] == snap2.shape[0]) & (snapshot.shape[0] == snap1.shape[0]) & (snap2.shape[0] == snap1.shape[0]))
        except:
            if snapshot.shape[0] == snap1.shape[0]:
                print("snap1 have more ticks than snap2")  
            if snapshot.shape[0] == snap2.shape[0]:
                print("pm_snap2 have more ticks than pm_snap1")
            else:
                print("pm_snap2 don't join with pm_snap1")
            snapshot = pd.merge(snap2, snap1, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"])
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print("深交所数据")
        

    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 200000)]
    snapshot.to_pickle(path1 + "\\Snapshot.pkl")
#     snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
#     s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
#     s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
#     s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
#     s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
#     s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
#     s2["ID"] = s2["ID"].astype(str)
#     s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
#     df += [s2]
#     print(s2.head())
    print(data + " finished~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# df = pd.concat(df).reset_index(drop=True)
# df.to_pickle(r"F:\\SZ2019_2.pkl")
# print(bad)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (51,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


am & pm data
J:\LEVEL2_shenzhen\2019\1225 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
am & pm data
J:\LEVEL2_shenzhen\2019\1226 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
am & pm data
J:\LEVEL2_shenzhen\2019\1227 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
am & pm data
J:\LEVEL2_shenzhen\2019\1230 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
am & pm data
J:\LEVEL2_shenzhen\2019\1231 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# Revised Version 2017-2018

In [3]:
columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]

startTm = datetime.datetime.now()
year = "2018"
df = []
bad = []
readPath = 'I:\\LEVEL2_shenzhen\\' + year + '\\***'

dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[1:2]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
        
    # am & pm
    if len(data) == 28:
        path = r'F:\SZ\2018' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()

        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data")
   
    # 深交所数据
    else:
        path = r'F:\SZ\2018' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\hq_snap.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\hq_snap.7z')
            bad.append(data + '\\hq_snap.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        
        snapshot = pd.read_table(path1 + "\\hq_snap.txt", header=None, encoding="UTF-8-sig")
        assert(snapshot.shape[1] == len(columns1))
        snapshot.columns = columns1
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("深交所数据")
        

    print(datetime.datetime.now() - startTm)
    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    print(datetime.datetime.now() - startTm)
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] <= 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    print(datetime.datetime.now() - startTm)
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2.groupby("StockID")["volume_update"].describe().reset_index(), on="StockID")
    data2["75%"] = data2["75%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 60) & (data2["volume_update"] >= data2["75%"])].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    print(datetime.datetime.now() - startTm)
    dd["auction"] = np.where(snapshot["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print(datetime.datetime.now() - startTm)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
display(df)
df.to_pickle(r"F:\\SZ2018.pkl")
print(bad)

20180103000000000
am & pm data
0:01:12.558114
0:01:14.010866
0:01:18.669489
0:07:12.802118
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2018-01-03   13.73     13.70   13.86  13.20    13.33  296249838   
1  SZ000002  2018-01-03   32.50     32.56   33.78  32.23    32.33   64687020   
2  SZ000004  2018-01-03   22.42     22.34   23.89  22.27    23.80    1879539   
3  SZ000005  2018-01-03    4.35      4.32    4.35   4.22     4.27    6799065   
4  SZ000006  2018-01-03    0.00      9.85    0.00   0.00     0.00          0   

       d_amount  massive_missing  security_missing  auction  
0  4.006221e+09                0                 0        1  
1  2.130250e+09                0                 0        1  
2  4.321842e+07                0                 0        1  
3  2.896679e+07                0                 0        1  
4  0.000000e+00                0                 0        0  
I:\LEVEL2_shenzhen\2018\0103 finished~
0:07:12.802118
~~

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2018-01-03,13.73,13.70,13.86,13.20,13.33,296249838,4.006221e+09,0,0,1
1,SZ000002,2018-01-03,32.50,32.56,33.78,32.23,32.33,64687020,2.130250e+09,0,0,1
2,SZ000004,2018-01-03,22.42,22.34,23.89,22.27,23.80,1879539,4.321842e+07,0,0,1
3,SZ000005,2018-01-03,4.35,4.32,4.35,4.22,4.27,6799065,2.896679e+07,0,0,1
4,SZ000006,2018-01-03,0.00,9.85,0.00,0.00,0.00,0,0.000000e+00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,SZ300729,2018-01-03,36.35,35.29,36.37,33.58,34.32,8565451,3.002981e+08,0,0,1
2071,SZ300730,2018-01-03,47.00,45.71,48.50,44.67,44.68,16409314,7.651693e+08,0,0,1
2072,SZ300731,2018-01-03,37.56,38.67,37.77,35.00,36.38,8095237,2.931803e+08,0,0,1
2073,SZ300732,2018-01-03,81.19,82.95,82.24,78.30,80.80,3999278,3.220272e+08,0,0,1


[]


In [10]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SZ' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    dayData = dayData[dayData["date"] == "2018-01-03"]
    db = pd.concat([db, dayData])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
df = pd.merge(df, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_close", "d_volume"], how="outer")
if sum(df["d_amount_x"].isnull()) != 0:
    diff = df[df["d_amount_x"].isnull()][["ID", "date"]]
print(diff)

            ID        date
2075  SZ001914  2018-01-03


In [12]:
F1 = open("F:\\SZ2018.pkl", 'rb')
d = pickle.load(F1)
d[(d["auction"] == 0) & (d["d_open"] != 0)]

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
445,SZ000982,2018-01-03,3.86,3.68,3.86,3.86,3.86,1313524,5070202.64,0,0,0
1363,SZ002921,2018-01-03,24.34,22.13,24.34,24.34,24.34,8600,209324.00,0,0,0
1364,SZ002922,2018-01-03,21.63,19.66,21.63,21.63,21.63,6771,146456.73,0,0,0
2030,SZ300684,2018-01-03,16.86,15.33,16.86,16.86,16.86,6629,111764.94,0,0,0
2074,SZ300735,2018-01-03,17.41,15.83,17.41,17.41,17.41,32190,560427.90,0,0,0


In [9]:
df[df["ID"] == "SZ300735"]

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
2074,SZ300735,2018-01-03,17.41,15.83,17.41,17.41,17.41,32190,560427.9,0,0,0


# Revised Version 2016

In [22]:
columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NUMTRADES", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "TOTALOFFERQTY", "WEIGHTEDAVGOFFERPX", "TOTALBIDQTY", "WEIGHTEDAVGBIDPX"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "NUMORDERS_B1", "NOORDERS _B1", "ORDERQTY_B1", "NUMORDERS_S1", "NOORDERS _S1", "ORDERQTY_S1"]


year = "2016"
df = []
bad = []
readPath = 'I:\\LEVEL2_shenzhen\\' + year + '\\***\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[1:2]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
    
    # SZ version A without AM PM, 分卷
    if len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001')[0]).split('.')[0].split('_')[2]
        os.mkdir(path + '\\' + d_ate)
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001')[0]).split('.')[0]
        os.system("copy /b " + n_ame + ".7z.* " + n_ame + ".7z")
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\***'))) == 1:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        for pp in np.array(glob.glob(path1 +'\\***')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot = pd.concat([snapshot, s1])
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A without AM PM")
        
    # SZ version A AM PM
    # I:\\LEVEL2_shenzhen\\2016\\04\\19\\SZL2_SNAPSHOT_20160419_AM.7z 58
    # I:\\LEVEL2_shenzhen\\2016\\05\\06\\SZL2_SNAPSHOT_20160506_AM.txt 59
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))) == 1:
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A AM PM, txt")
                
        
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z')[0]).split('.')[0].split('_')[2]
        os.mkdir(path + '\\' + d_ate)
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        n_ame1 = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame1 + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame1 + '.7z')
            bad.append(data + "\\" + n_ame1 + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"] 
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A AM PM, .7z")
 
    # am & pm
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\09\\am_snap_level_spot.7z'
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\10\\am_snap_level_spot.7z.001'
    
    elif len(np.array(glob.glob(data +'\\am_hq_snap_spot.7z'))) == 1:
        path = r'F:\SZ\2016' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 未分卷")
        
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z.001'))) == 1:    
        path = r'F:\SZ\2016' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
      
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1
        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 分卷")
   
  

    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] <= 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])    
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2.groupby("StockID")["volume_update"].describe().reset_index(), on="StockID")
    data2["75%"] = data2["75%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 60) & (data2["volume_update"] >= data2["75%"])].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    dd["auction"] = np.where(snapshot["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
df.to_pickle(r"F:\\SZ2016.pkl")
print(bad)

SZ version A without AM PM


AssertionError: 

In [20]:
d2.shape[0]

1554

In [5]:
np.array(glob.glob(path1 +'\\***'))[0]

'F:\\SZ\\2016\\20160104\\SZL2_SNAPSHOT_20160104_AM.txt'

In [57]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
am_snap1 = pd.read_table("F:\\SZ\\2019\\0102\\am_hq_snap_spot.txt", header=None)
am_snap1.columns = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
am_snap2 = pd.read_table("F:\\SZ\\2019\\0102\\am_snap_level_spot.txt",header=None)
am_snap2.columns = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]

In [63]:
re = pd.merge(am_snap2, am_snap1, on=["Date","OrigTime","SendTime","ercvtime","dbtime", "ChannelNo","SecurityID","SecurityIDsource", "MDStreamID"], how="outer")
display(am_snap2.shape[0])
display(re.shape[0])
display(am_snap1.shape[0])


4088148

4183008

4088148

In [45]:
test[test["TradingPhase"] == "E0"]

,Date,OrigTime,SendTime,ercvtime,dbtime,ChannelNo,SecurityID,SecurityIDsource,MDStreamID,PreClosePx,PxChnage1,PXChange2,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,PE1,PE2,TradingPhase,totalofferqty,wa_offerPrice,totalbidqty,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,unknown1,unknown2,unknown3,high1,low1
3491467,20190102,20190102150003000,20190102150003911,20190102150004100,20190102150004103,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3497726,20190102,20190102150103000,20190102150103853,20190102150104518,20190102150104520,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3504309,20190102,20190102150203000,20190102150203915,20190102150203924,20190102150203926,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3510730,20190102,20190102150303000,20190102150303901,20190102150303912,20190102150303913,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3517151,20190102,20190102150403000,20190102150403898,20190102150403910,20190102150403910,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3523572,20190102,20190102150503000,20190102150503903,20190102150503912,20190102150503913,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3529993,20190102,20190102150603000,20190102150603924,20190102150603933,20190102150603934,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3536414,20190102,20190102150703000,20190102150703921,20190102150703930,20190102150703931,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3542679,20190102,20190102150803000,20190102150803841,20190102150803851,20190102150803852,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41
3549256,20190102,20190102150903000,20190102150903908,20190102150903917,20190102150903918,1012,300001,102,10,17.53,0.0,0.0,17.53,17.82,17.41,17.53,7884,5199500,91578660.13,62.83,0.0,E0,984495.0,18.23,424300.0,17.12,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.41


In [39]:
test = am_snap1[(am_snap1["SecurityID"] == 300001) & (am_snap1["cum_volume"] > 0)]
test["high1"] = test["HighPx"].cummax()
test["low1"] = test["LowPx"].cummin()
display(test[test["high1"] != test["HighPx"]])
display(test[test["low1"] != test["LowPx"]])
test.head(20)

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Date,OrigTime,SendTime,ercvtime,dbtime,ChannelNo,SecurityID,SecurityIDsource,MDStreamID,PreClosePx,PxChnage1,PXChange2,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,PE1,PE2,TradingPhase,totalofferqty,wa_offerPrice,totalbidqty,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,unknown1,unknown2,unknown3,high1,low1


,Date,OrigTime,SendTime,ercvtime,dbtime,ChannelNo,SecurityID,SecurityIDsource,MDStreamID,PreClosePx,PxChnage1,PXChange2,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,PE1,PE2,TradingPhase,totalofferqty,wa_offerPrice,totalbidqty,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,unknown1,unknown2,unknown3,high1,low1


,Date,OrigTime,SendTime,ercvtime,dbtime,ChannelNo,SecurityID,SecurityIDsource,MDStreamID,PreClosePx,PxChnage1,PXChange2,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,PE1,PE2,TradingPhase,totalofferqty,wa_offerPrice,totalbidqty,wa_bidPrice,PreNAV,RealTimeNAV,WarrantPremiumRate,UpLimitPx,DownLimitPx,TotalLongPosition,unknown1,unknown2,unknown3,high1,low1
5596,20190102,20190102124503000,20190102124503330,20190102124503492,20190102124503494,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
12266,20190102,20190102124603000,20190102124603366,20190102124603507,20190102124603510,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
18653,20190102,20190102124703000,20190102124703325,20190102124703525,20190102124703527,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
25099,20190102,20190102124803000,20190102124803338,20190102124803680,20190102124803691,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
31525,20190102,20190102124903000,20190102124903342,20190102124903469,20190102124903471,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
37707,20190102,20190102125003000,20190102125003294,20190102125003578,20190102125003580,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
43926,20190102,20190102125103000,20190102125103273,20190102125103570,20190102125103571,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
50759,20190102,20190102125203000,20190102125203319,20190102125203618,20190102125203621,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
57210,20190102,20190102125303000,20190102125303345,20190102125303370,20190102125303372,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51
63504,20190102,20190102125403000,20190102125403285,20190102125403473,20190102125403475,1012,300001,102,10,17.53,0.13,-0.02,17.53,17.82,17.51,17.66,4757,3256400,57528694.42,63.30,0.0,B0,823895.0,18.37,630457.0,17.29,0.0,0.0,0.0,19.28,15.78,0.0,0.0,0.0,0.0,17.82,17.51


In [9]:
am_snap1[am_snap1["SecurityID"].isin(stockSZ)]["ChannelNo"].unique()

array([1014, 1011, 1013, 1012], dtype=int64)

In [28]:
# KK = pd.DataFrame(stockSZ, columns=["stock"])
KK[(KK["stock"] > 300000)].min()

stock    300001
dtype: int64

In [57]:
am_snap1[(am_snap1["SecurityID"] < 3000) & (am_snap1["SecurityID"] > 4000) ]["SecurityID"].unique()

array([], dtype=int64)